In [ ]:
# Check Colab Pro / Local instance specs
# !df -h
# !cat /proc/cpuinfo
# !cat /proc/meminfo

In [1]:
import os
import re
import gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
%pip install fuzzywuzzy
%pip install python-Levenshtein
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

%pip install rapidfuzz # Run pip install if using Google Colab
import rapidfuzz
from rapidfuzz import process as process, fuzz
from rapidfuzz.process import cdist, extract

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 9.6 MB/s eta 0:00:00


# Setting up Google Drive access if not running locally

In [3]:
# Mount Google Drive. Confirm access permissions to permit this notebook to access your Google Drive files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# change proj_gdrive_dir string to personal gdrive project code path

#proj_dir = '/content/drive/MyDrive/' + 'datasci-210/datasci-210-project/fraud-detect/'
#for Mike
proj_dir = '/content/drive/MyDrive/' + 'datasci-210-project/fraud-detect/'
proj_dir_data_raw = proj_dir + 'data/raw/' # Assign data folder under project code path. Create separately in Gdrive
proj_dir_data_proc = proj_dir + 'data/processed/'
!ls $proj_dir
print('-----------------------------------------------------------------------')
!ls $proj_dir_data_raw
print('-----------------------------------------------------------------------')
!ls $proj_dir_data_proc

data  notebooks
-----------------------------------------------------------------------
ppp-borrower-loanid.csv  ppp-data-combined.csv
-----------------------------------------------------------------------
case_data.xlsx	out_150.gsheet	out_300.gsheet	     out_400_Onwards.gsheet  out.gsheet
out_150.csv	out_300.csv	out_400_Onwards.csv  out.csv


In [ ]:
# Run if the Google drive needs to be force remounted because updated files are not visible on Colab
# drive.mount("/content/drive", force_remount=True)

# Run after completing Notebook run to gracefully disconnect Gdrive
# drive.flush_and_unmount()

# Loading locally

In [ ]:
# %pwd

'/content'

In [ ]:
# Set if running on a local isntance and don't run Gdrive set-up
# proj_dir = '/home/cs/Documents/fraud-detect/'
# proj_dir_data_raw = proj_dir + 'data/raw/'
# proj_dir_data_proc = proj_dir + 'data/processed/'

# Create new borrower name and loan id file from PPP data file

In [ ]:
dtype_dict = {
    'LoanNumber':'str',
    'DateApproved':'str',
    'SBAOfficeCode':'str',
    'ProcessingMethod':'str',
    'BorrowerName':'str',
    'BorrowerAddress':'str',
    'BorrowerCity':'str',
    'BorrowerState':'str',
    'BorrowerZip':'str',
    'LoanStatusDate':'str',
    'LoanStatus':'str',
    # 'Term':'Int',
    # 'SBAGuarantyPercentage':'int',
    'InitialApprovalAmount':'float',
    'CurrentApprovalAmount':'float',
    'UndisbursedAmount':'float',
    'FranchiseName':'str',
    'ServicingLenderLocationID':'str',
    'ServicingLenderName':'str',
    'ServicingLenderAddress':'str',
    'ServicingLenderCity':'str',
    'ServicingLenderState':'str',
    'ServicingLenderZip':'str',
    'RuralUrbanIndicator':'str',
    'HubzoneIndicator':'str',
    'LMIIndicator':'str',
    'BusinessAgeDescription':'str',
    'ProjectCity':'str',
    'ProjectCountyName':'str',
    'ProjectState':'str',
    'ProjectZip':'str',
    'CD':'str',
    # 'JobsReported':'int',
    'NAICSCode':'str',
    'Race':'str',
    'Ethnicity':'str',
    'UTILITIES_PROCEED':'float',
    'PAYROLL_PROCEED':'float',
    'MORTGAGE_INTEREST_PROCEED':'float',
    'RENT_PROCEED':'float',
    'REFINANCE_EIDL_PROCEED':'float',
    'HEALTH_CARE_PROCEED':'float',
    'DEBT_INTEREST_PROCEED':'float',
    'BusinessType':'str',
    'OriginatingLenderLocationID':'str',
    'OriginatingLender':'str',
    'OriginatingLenderCity':'str',
    'OriginatingLenderState':'str',
    'Gender':'str',
    'Veteran':'str',
    'NonProfit':'str',
    'ForgivenessAmount':'float',
    'ForgivenessDate':'str'
}

parse_dates = ['DateApproved', 'LoanStatusDate', 'ForgivenessDate']

In [ ]:
# # Load SBA PPP Combined file
filename = 'ppp-data-combined.csv'
df_ppp_data = pd.read_csv(proj_dir_data_raw+filename, header=0, dtype=dtype_dict, parse_dates=parse_dates)


KeyboardInterrupt: ignored

In [ ]:
cols = ['LoanNumber', 'BorrowerName', 'InitialApprovalAmount']
df_ppp_data = df_ppp_data.dropna(subset=cols)
df_ppp_data.shape

In [ ]:
filename = 'ppp-borrower-loanid.csv'
cols = ['LoanNumber','BorrowerName']
df_ppp_data.to_csv(proj_dir_data_raw+filename, columns=cols, index=False)

# Load raw Loan Number and Borrower Name file

In [7]:
filename = 'ppp-borrower-loanid.csv'
df_borrower = pd.read_csv(proj_dir_data_raw+filename, header=0)
df_borrower.shape

(11765501, 2)

In [8]:
# Modified from finance paper code
df_borrower['BorrowerNameProc'] = df_borrower['BorrowerName']\
                                        .str.lower()\
                                        .str.replace('[^a-zA-Z0-9 ]', '')\
                                        .str.replace('(?<!\S)incorporated(?=\s|$)', '')\
                                        .str.replace('(?<!\S)llc(?=\s|$)', ' ')\
                                        .str.replace('(?<!\S)corporation(?=\s|$)', '')\
                                        .str.replace('(?<!\S)company(?=\s|$)', '')\
                                        .str.replace('(?<!\S)partnership(?=\s|$)', '')\
                                        .str.replace('(?<!\S)limited(?=\s|$)', '')\
                                        .str.replace('(?<!\S)inc(?=\s|$)', '')\
                                        .str.replace('(?<!\S)co(?=\s|$)', '')\
                                        .str.replace('(?<!\S)lc(?=\s|$)', '')\
                                        .str.replace('(?<!\S)pllc(?=\s|$)', '')\
                                        .str.replace('(?<!\S)llp(?=\s|$)', '')\
                                        .str.replace('\s\s+', '')\
                                        .str.strip()

<ipython-input-8-249a4889e02b>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  .str.replace('[^a-zA-Z0-9 ]', '')\
<ipython-input-8-249a4889e02b>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  .str.replace('(?<!\S)incorporated(?=\s|$)', '')\
<ipython-input-8-249a4889e02b>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  .str.replace('(?<!\S)llc(?=\s|$)', ' ')\
<ipython-input-8-249a4889e02b>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  .str.replace('(?<!\S)corporation(?=\s|$)', '')\
<ipython-input-8-249a4889e02b>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  .str.replace('(?<!\S)company(?=\s|$)', '')\
<ipython-input-8-249a4889e02b>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  .st

# Loading the labeled prosecuted case data

In [9]:
filename = 'case_data.xlsx'
cols = ['State','Lookup','Fraud','Status']
df_case_data = pd.read_excel(proj_dir_data_proc+filename, usecols=cols, header=0)
df_case_data.shape

(108, 4)

In [ ]:
df_case_data.head()

,State,Lookup,Fraud,Status
0,Arizona,Sam Fielder;Austin VanScoyk;Johnathan VanScoyk...,1,NaN
1,Arizona,Sam Fielder;Austin VanScoyk;Johnathan VanScoyk...,1,NaN
2,Arizona,Sam Fielder;Austin VanScoyk;Johnathan VanScoyk...,1,NaN
3,California,Mark Schena;Arrayit Corporation,1,NaN
4,California,Gurgen Israyelyan;Saint Christopher Hospice In...,1,NaN


In [10]:
# Modified from finance paper code
df_case_data['lookupproc'] = df_case_data['Lookup']\
                                        .str.lower()\
                                        .str.replace('[^a-zA-Z0-9; ]', '')\
                                        .str.replace('incorporated', '')\
                                        .str.replace(' llc', '')\
                                        .str.replace('corporation', '')\
                                        .str.replace('company', '')\
                                        .str.replace('partnership', '')\
                                        .str.replace('limited', '')\
                                        .str.replace(' inc', '')\
                                        .str.replace(' co', '')\
                                        .str.replace(' lc', '')\
                                        .str.replace(' pllc', '')\
                                        .str.replace(' llp', '')\
                                        .str.replace('\s\s+', '')\
                                        .str.strip()

# Split the lookup column string into list of strings for each case
df_case_data['lookup_li'] = df_case_data['lookupproc'].str.split(';', expand=False)
# Explode the rows based on the lookup list
print("Shape before:", df_case_data.shape)
df_case_data = df_case_data.explode('lookup_li')
# Set index to tie back to cases and reindex
df_case_data['case_index'] = df_case_data.index
df_case_data = df_case_data.reset_index()
print("Shape after:", df_case_data.shape)

Shape before: (108, 6)
Shape after: (614, 8)


<ipython-input-10-198655539f6c>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  .str.replace('[^a-zA-Z0-9; ]', '')\
<ipython-input-10-198655539f6c>:16: FutureWarning: The default value of regex will change from True to False in a future version.
  .str.replace('\s\s+', '')\


# Match Labeled to Loan Data

In [ ]:
df_case_data['lookup_li'][3]

'bright edge'

In [ ]:
# Test with known business name
thresh = 95 # Set threshold for score cutoff when running extract
out = process.extract(df_case_data['lookup_li'][3], df_borrower['BorrowerNameProc'], limit=10, score_cutoff=thresh)
print(df_case_data['lookup_li'][3], '\n', out)

bright edge 
 [('bright edge', 100.0, 7222483)]


In [ ]:
df_borrower.loc[df_borrower['BorrowerNameProc'] == 'bright edge']

,LoanNumber,BorrowerName,BorrowerNameProc
7222483,8820467005,BRIGHT EDGE LLC,bright edge


In [ ]:
import multiprocessing
multiprocessing.cpu_count()
# https://mlexplained.blog/tag/fuzze-merge/

8

In [ ]:
#Fuzzywuzzy Merge from: https://stackoverflow.com/questions/13636848/is-it-possible-to-do-fuzzy-match-merge-with-python-pandas
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=100):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()

    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))
    df_1['matches'] = m

    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2

    return df_1

In [ ]:
import time
rows = 10
#df_case_data.iloc[:rows]

t0 = time.time()
test = fuzzy_merge(df_borrower, df_case_data.iloc[:rows], 'BorrowerNameProc', 'lookup_li', threshold = 95, limit = 100)
t1 = time.time()
print(rows,'rows took:', t1-t0, 'secs')
#1 rows took: 80.14567422866821 secs
#10 rows took: 337.62549924850464 secs
#100 rows took: 2919.09898853302 secs
#all rows took: 18526.394767284393 secs

10 rows took: 408.82354712486267 secs


In [ ]:
filtered = test[test['matches'] != '']
filtered

,LoanNumber,BorrowerName,BorrowerNameProc,matches
113176,2819028309,EASTON HOMES INC.,easton homes,aston homes
119041,1129397110,"EASTON HOMES, INC.",easton homes,aston homes
524586,1129308504,AMERICAN HOMES INCORPORATED,american homes,grand american homes
524857,8703267103,AMERICAN HOMES INCORPORATED,american homes,grand american homes
1856159,4649818403,"ASHTON HOMES, L.L.C.",ashton homes,aston homes
1906557,3456488802,AMERICAN HOMES LLC,american homes,grand american homes
2019037,2501927203,AMERICAN HOMES LLC,american homes,grand american homes
4194392,3801047707,AMERICAN HOMES LLC,american homes,grand american homes
4342704,3110787206,ASHTON HOMES INC,ashton homes,aston homes
4342705,3115858503,ASHTON HOMES INC.,ashton homes,aston homes


In [ ]:
%pip install python-Levenshtein
%pip install fuzzy_pandas
import fuzzy_pandas as fpd

#problem between 355-400

results = fpd.fuzzy_merge(df_borrower, df_case_data.iloc[355:360],
            left_on = 'BorrowerNameProc',
            right_on = 'lookup_li',
            method = 'levenshtein',
            threshold = 0.95)

results.to_csv(proj_dir_data_proc+'out.csv')

ZeroDivisionError: ignored

# Regular Merge

In [12]:
#Just do a regular inner merge
results = df_borrower.merge(df_case_data, left_on = 'BorrowerNameProc', right_on = 'lookup_li')
results.to_csv(proj_dir_data_proc+'labelled_data.csv')
#Unique loan numbers
print("number of matches:", results.shape[0])
print("number of unique loan numbers:", results.LoanNumber.nunique())
print("number of unique cases:", results.case_index.nunique())

In [ ]:
df_borrower['join_key_tuple'] = df_borrower['BorrowerNameProc'].apply(lambda x: extract(query = x, choices = df_case_data['lookup_li'], score_cutoff = thresh))
df_case_data.head()

In [ ]:
results.to_csv(proj_dir_data_proc+'out.csv')

In [ ]:
# Run finance paper buinsess name clean up on defendant / business name list from case data


In [ ]:
# re.sub(regex_search_term, regex_replacement, str_input)